# GP

What MSE does the GP achieve

In [1]:
import torch

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel
from gpytorch.distributions import MultivariateNormal

from os.path import join

import sys
sys.path.append('../')
import utils

In [2]:
assert torch.cuda.is_available(), 'CUDA is not available.'
#print('Runs on CPU, not enough memory on GPU')

## Load Data

In [3]:
fname = join('/home/squirt/Documents/data/weather_data/', 'all_data.h5')

In [4]:
split = 0.5
train_data, test_data = utils.get_data(fname, split)

In [5]:
def combine_data(data_tuple:tuple[torch.tensor]) -> tuple[torch.tensor]:
    '''
    Add Landmass to x data. Return x,y tensors
    Input:
        - data_tuple (tuple[torch.tensor]): tuple of landmass,x,y tensors
    '''
    l,x,y = data_tuple

    # Combine
    l = l.unsqueeze(1)
    x = torch.cat((l, x), 1)
    x = x.contiguous()
    y = y.contiguous()
    return (x,y)

In [6]:
train_data = combine_data(train_data)
test_data = combine_data(test_data)

In [7]:
x,y = train_data
x = x.view(-1, 71*3*2*2)
y = y.view(-1, 70*2*2*2)

#'''
x = x.float().cuda()
y = y.float().cuda()
'''
x = x.float()
y = y.float()
#'''

'\nx = x.float()\ny = y.float()\n#'

In [8]:
x_test, y_test = test_data
x_test = x_test.view(-1, 71*3*2*2)
y_test = y_test.view(-1, 70*2*2*2)

#'''
x_test = x_test.float().cuda()
y_test = y_test.float().cuda()
'''
x_test = x_test.float()
y_test = y_test.float()
#'''

'\nx_test = x_test.float()\ny_test = y_test.float()\n#'

## Define GP

Define Model using gptorch

In [9]:
class SparseMultitaskGaussianModel(gpytorch.models.ApproximateGP):
    def __init__(self, train_x, train_y, likelihood):
        inducing_points = train_x[:100]
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(0)
        )
        variational_strategy = gpytorch.variational.IndependentMultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=train_y.shape[-1]
        )
        super(SparseMultitaskGaussianModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=train_y.shape[-1]
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=train_y.shape[-1], rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


In [10]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=y.shape[1])
likelihood = likelihood.cuda()

model = SparseMultitaskGaussianModel(x, y, likelihood=likelihood)
#model = model.float()
model = model.float().cuda()

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
mll = mll.cuda()

## Train Model

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [12]:
def eval(model, x, y):
    model.eval()
    batch_size = 50 

    # Create a DataLoader for your test data
    test_dataset = torch.utils.data.TensorDataset(x, y)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    total_loss = 0.
    with torch.no_grad():
        for xb, yb in test_loader:  
            output = model(xb).mean
            loss = torch.nn.functional.mse_loss(output, yb, reduction='sum')
            total_loss += loss.item()

    mse = total_loss / len(test_loader.dataset)
    return mse

In [13]:
# Training loop
likelihood.train()

for epoch in range(10):
    # Train Model
    model.train()
    optimizer.zero_grad()

    # This is a key part: call your model on a batch
    output = model(x)
    loss = -mll(output, y)
    loss.backward()
        
    optimizer.step()

    x = x.cpu()
    y = y.cpu()

    #Test Model
    #model.eval()
    # Move to batches so memory doesn't blow up to 250 TB
    #loss = torch.nn.functional.mse_loss(model(x_test).mean, y_test)
    loss = eval(model, x_test, y_test)
    print(f'Epoch {epoch} - Loss: {loss}')

/home/squirt/.my_conda/envs/climate/lib/python3.12/site-packages/linear_operator/utils/interpolation.py:71: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/torch/csrc/tensor/python_tensor.cpp:78.)
  summing_matrix = cls(summing_matrix_indices, summing_matrix_values, size)
/home/squirt/.my_conda/envs/climate/lib/python3.12/site-packages/linear_operator/utils/interpolation.py:71: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/torch/csrc/utils/tensor_new.cpp:621.)
  summing_matrix = cls(summing_matrix_indices, summing_matrix_values, size)


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.03 GiB. GPU 